In [ ]:
# Define the output file name
output_file = "consolidated_data.csv"

# Initialize a list to store merged DataFrames from all subfolders
all_subfolder_data = []

# Traverse through all subfolders in the current directory
for root, dirs, files in os.walk(os.getcwd()):
    infotable_df = None
    coverpage_df = None
    print(root)

    for file in files:
        file_path = os.path.join(root, file)

        # Process INFOTABLE.tsv
        if file == "INFOTABLE.tsv":
            try:
                print(f"Processing: {file_path}")
                infotable_df = pd.read_csv(file_path, sep="\t", low_memory=False)
            except Exception as e:
                print(f"Error processing {file_path}: {e}")

        # Process COVERPAGE.tsv
        if file == "COVERPAGE.tsv":
            try:
                print(f"Processing: {file_path}")
                coverpage_df = pd.read_csv(file_path, sep="\t", low_memory=False)
                # Keep only necessary columns
                coverpage_df = coverpage_df[["ACCESSION_NUMBER", "FILINGMANAGER_NAME", "DATEREPORTED"]]
            except Exception as e:
                print(f"Error processing {file_path}: {e}")

    # Merge INFOTABLE and COVERPAGE on ACCESSION_NUMBER if both are present
    if infotable_df is not None and coverpage_df is not None:
        try:
            merged_subfolder_data = pd.merge(infotable_df, coverpage_df, on="ACCESSION_NUMBER", how="inner")
            all_subfolder_data.append(merged_subfolder_data)
        except Exception as e:
            print(f"Error merging files in {root}: {e}")

# Combine all merged subfolder DataFrames into a single DataFrame
if all_subfolder_data:
    consolidated_data = pd.concat(all_subfolder_data, ignore_index=True)
    consolidated_data.to_csv(output_file, index=False)
    print(f"Consolidation complete. File saved as {output_file}")
else:
    print("No data to process.")
    



In [ ]:
# Import the pandas library
import pandas as pd

# Function to check and print the fields of a CSV file
def check_csv_fields(file_path):
    """
    Load a CSV file and print its fields (column names) along with metadata.

    Args:
        file_path (str): Path to the CSV file.
    """
    try:
        # Load CSV into a DataFrame without loading all data into memory
        df = pd.read_csv(file_path, nrows=0)

        # Print column names
        print("Field (Column) Names:")
        for column in df.columns:
            print(f"- {column}")

        # Print metadata
        print(f"\nNumber of fields: {len(df.columns)}")

    except Exception as e:
        print(f"Error loading CSV file: {e}")

# Example usage
# Replace 'your_file.csv' with the path to your CSV file
check_csv_fields('consolidated_data_2.csv')


In [ ]:
import pandas as pd

def delete_fields_from_csv(input_file, output_file, fields_to_delete):
    """
    Deletes specified fields (columns) from a CSV file and saves the updated data to a new file.

    Args:
        input_file (str): Path to the input CSV file.
        output_file (str): Path to save the updated CSV file.
        fields_to_delete (list): List of column names to delete from the CSV.

    Returns:
        None
    """
    try:
        # Load the CSV file
        df = pd.read_csv(input_file)

        # Check if the columns to delete exist in the DataFrame
        missing_fields = [field for field in fields_to_delete if field not in df.columns]
        if missing_fields:
            print(f"Warning: The following fields were not found in the CSV and will be ignored: {missing_fields}")

        # Drop the specified fields
        updated_df = df.drop(columns=fields_to_delete, errors='ignore')

        # Save the updated DataFrame to a new CSV file
        updated_df.to_csv(output_file, index=False)
        print(f"Updated CSV saved to '{output_file}'.")

    except Exception as e:
        print(f"Error processing the file: {e}")

# Example usage
input_file = 'consolidated_data_2.csv'  # Replace with your input file path
output_file = 'final_13f.csv'  # Replace with your desired output file path
fields_to_delete = ['ACCESSION_NUMBER']  # Replace with the fields/columns you want to delete

delete_fields_from_csv(input_file, output_file, fields_to_delete)

In [ ]:
import pandas as pd

class YourClass:
    input_path = "final_13f.csv"

    def CUSIP_to_ticker(self, output_csv_path="institutional_data.csv"):
        """
        Reads institutional data, maps CUSIP codes to tickers using a reference CSV,
        removes invalid dates, and writes the resulting DataFrame to a CSV file.

        Args:
            output_csv_path (str): Path to save the resulting CSV file.

        Returns:
            str: Path to the generated CSV file.
        """
        # Reference CSV for CUSIP-to-Ticker mapping
        df_ref = pd.read_csv("cusip_tickers.csv", dtype=str)

        # Check if required columns exist
        if not {"CUSIP", "Ticker"}.issubset(df_ref.columns):
            raise ValueError("Reference CSV must have 'CUSIP' and 'Ticker' columns.")

        # Create a mapping dictionary
        cusip_to_ticker = df_ref.set_index("CUSIP")["Ticker"].to_dict()

        original_csv = self.input_path  # Path to the input file
        chunksize = 10**5  # Process data in chunks of 100,000 rows

        # Open the output file in write mode
        with pd.read_csv(original_csv, dtype=str, chunksize=chunksize) as reader, open(output_csv_path, "w") as output_file:
            for chunk_index, chunk in enumerate(reader):
                print(f"Processing chunk {chunk_index + 1}...")

                # Validate required columns
                if "CUSIP" not in chunk.columns or "DATEREPORTED" not in chunk.columns:
                    raise ValueError("Input CSV is missing 'CUSIP' or 'DATEREPORTED' columns.")

                # Map CUSIP to Ticker
                chunk["CUSIP"] = chunk["CUSIP"].map(cusip_to_ticker).fillna(chunk["CUSIP"])

                # Rename 'CUSIP' column to 'Ticker'
                chunk.rename(columns={"CUSIP": "Ticker", "DATEREPORTED": "date"}, inplace=True)

                # Convert 'date' column to datetime and drop invalid dates
                chunk["date"] = pd.to_datetime(chunk["date"], errors="coerce")
                chunk.dropna(subset=["date"], inplace=True)  # Remove rows with invalid dates

                # Debugging: Print the first few rows of the current chunk
                print(chunk.head())

                # Write the processed chunk to the output CSV
                mode = "w" if chunk_index == 0 else "a"  # Write header only for the first chunk
                header = chunk_index == 0
                chunk.to_csv(output_file, index=False, mode=mode, header=header)

        print(f"Consolidated data has been saved to {output_csv_path}")
        return output_csv_path


# Create an instance of YourClass and call the method
analyzer = YourClass()
output_csv_path = analyzer.CUSIP_to_ticker()
print(f"Output CSV generated at: {output_csv_path}")


In [ ]:
# Old code

"""
df_ref = pd.read_csv("cusip_tickers.csv", dtype=str)
cusip_to_ticker = df_ref.set_index("CUSIP")["Ticker"].to_dict()

# Check columns exist
if not {"CUSIP", "Ticker"}.issubset(df_ref.columns):
    raise ValueError("Reference CSV must have 'CUSIP' and 'Ticker' columns.")
    
# Convert to dict
   
def process_dataframe(df, cusip_to_ticker):
    """
    Maps CUSIP codes to tickers within a DataFrame and renames the column.

    Args:
        df (pd.DataFrame): The original DataFrame with a 'CUSIP' column.
        cusip_to_ticker (dict): Dictionary mapping CUSIP codes to tickers.

    Returns:
        pd.DataFrame: The updated DataFrame with 'CUSIP' replaced by 'Ticker'.
    """
    # Check if "CUSIP" exists in the DataFrame
    if "CUSIP" not in df.columns:
        raise ValueError("The DataFrame is missing a 'CUSIP' column.")

    # Map CUSIP to Ticker and handle missing mappings
    df["CUSIP"] = df["CUSIP"].map(cusip_to_ticker).fillna(df["CUSIP"])

    # Rename "CUSIP" column to "Ticker"
    df.rename(columns={"CUSIP": "Ticker"}, inplace=True)

    return df


# Process the DataFrame
processed_df = process_dataframe(df, cusip_to_ticker)

print(processed_df)
      
    """
    

In [ ]:
import os
import pandas as pd
from sec_cik_mapper import StockMapper

mapper = StockMapper()
# Define the parent directory to process
def process_googl_holdings(parent_directory):
    result_data = []  # To store filtered and aggregated data

    # Define GOOGL CUSIP
    googl_cusip1 = "02079K305"
    googl_cusip2 = "02079K107"

    # Walk through all subfolders
    for root, dirs, files in os.walk(parent_directory):
        info_table_path = None
        submission_path = None

        # Identify required files in the current directory
        for file in files:
            if file.lower() == "infotable.tsv":
                info_table_path = os.path.join(root, file)
            elif file.lower() == "submission.tsv":
                submission_path = os.path.join(root, file)

        # If both files are found, process them
        if info_table_path and submission_path:
            # Load INFO TABLE
            try:
                info_table_df = pd.read_csv(info_table_path, sep="\t")
                # Filter for GOOGL holdings
                googl_holdings1 = info_table_df[info_table_df["CUSIP"] == googl_cusip1]
                googl_holdings2 = info_table_df[info_table_df["CUSIP"] == googl_cusip2]
                googl_holdings = pd.concat([googl_holdings1, googl_holdings2], ignore_index=True)
                # print(info_table_df)
                if googl_holdings.empty:
                    continue  # Skip if no GOOGL holdings found
            except Exception as e:
                print(f"Error reading INFO TABLE at {info_table_path}: {e}")
                continue

            # Load SUBMISSION
            try:
                submission_df = pd.read_csv(submission_path, sep="\t")
                # print(submission_df)
                # Join INFO TABLE with SUBMISSION on accession number
                if "ACCESSION_NUMBER" in info_table_df.columns and "ACCESSION_NUMBER" in submission_df.columns:
                    googl_holdings = googl_holdings.merge(
                        submission_df, on="ACCESSION_NUMBER", how="inner"
                    )
            except Exception as e:
                print(f"Error reading SUBMISSION at {submission_path}: {e}")
                continue

            # Combine GOOGL holdings with submission metadata
            try:
                for _, row in googl_holdings.iterrows():
                    combined_data = {
                        "issuer_name": row.get("NAMEOFISSUER"),
                        "cusip": row.get("CUSIP"),
                        "value": row.get("VALUE"),
                        "shares": row.get("SSHPRNAMT"),
                        "share_type": row.get("SSHPRNAMTTYPE"),
                        "put_call": row.get("PUTCALL", None),
                        "submission_cik": row.get("CIK"),
                        "submission_ticker": mapper.cik_to_tickers.get(str(row.get("CIK")).zfill(10), None),
                        "submission_form_type": row.get("SUBMISSIONTYPE"),
                        "submission_period": row.get("PERIODOFREPORT"),
                        "submission_filing_date": row.get("FILING_DATE"),
                    }
                    result_data.append(combined_data)

            except Exception as e:
                print(f"Error processing data from {info_table_path} and {submission_path}: {e}")
                continue

    # Create final DataFrame
    result_df = pd.DataFrame(result_data)

    # Save the filtered and aggregated data to a CSV file
    output_path = os.path.join(parent_directory, "googl_holdings_aggregated.csv")
    result_df.to_csv(output_path, index=False)
    print(f"Filtered GOOGL holdings data saved to {output_path}")

# Specify the parent directory (replace this with the actual path)
parent_directory = "/Users/lockin/Downloads/"
process_googl_holdings(parent_directory)


/var/folders/26/yfv_whfd2pv776k5pk93tvt80000gn/T/ipykernel_4738/3946070976.py:30: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  info_table_df = pd.read_csv(info_table_path, sep="\t")
/var/folders/26/yfv_whfd2pv776k5pk93tvt80000gn/T/ipykernel_4738/3946070976.py:30: DtypeWarning: Columns (5,11) have mixed types. Specify dtype option on import or set low_memory=False.
  info_table_df = pd.read_csv(info_table_path, sep="\t")


In [ ]:

mapper.cik_to_tickers

# Function to convert CIKs to tickers
def convert_cik_to_ticker(df, cik_column):
    # Initialize the StockMapper
    mapper = StockMapper()
    cik_to_ticker = mapper.cik_to_tickers  # Dictionary mapping CIK to tickers

    # Add a new column for the ticker
    df['cik_ticker'] = df[cik_column].apply(lambda cik: cik_to_ticker.get(str(cik).zfill(10), None))

    return df

{'0000001750': {'AIR'},
 '0000001800': {'ABT'},
 '0000001961': {'WDDD'},
 '0000002098': {'ACU'},
 '0000002178': {'AE'},
 '0000002186': {'BKTI'},
 '0000002230': {'ADX'},
 '0000002488': {'AMD'},
 '0000002809': {'AEM'},
 '0000002969': {'APD'},
 '0000003197': {'CECO'},
 '0000003453': {'MATX'},
 '0000003499': {'ALX'},
 '0000003545': {'ALCO'},
 '0000003570': {'LNG'},
 '0000004127': {'SWKS'},
 '0000004281': {'HWM', 'HWM-P'},
 '0000004447': {'HES'},
 '0000004457': {'UHAL', 'UHAL-B'},
 '0000004904': {'AEP'},
 '0000004962': {'AXP'},
 '0000004977': {'AFL'},
 '0000005094': {'VBF'},
 '0000005272': {'AIG'},
 '0000005513': {'UNM', 'UNMA'},
 '0000005981': {'AVD'},
 '0000006176': {'AP', 'AP-WT'},
 '0000006201': {'AAL'},
 '0000006207': {'AXR'},
 '0000006281': {'ADI'},
 '0000006845': {'APOG'},
 '0000006951': {'AMAT'},
 '0000006955': {'EPAC'},
 '0000007084': {'ADM'},
 '0000007323': {'EAI'},
 '0000007431': {'AWI'},
 '0000007536': {'ARW'},
 '0000007623': {'ARTW'},
 '0000007789': {'ASB', 'ASB-PE', 'ASB-PF', 